# Get total list of patients

In [ ]:
import glob
import os
import random
import shutil

randseed = random.randint(1, 1000)
print("randseed: ", randseed)
rng = random.Random(randseed)

data_category = ["SNUH_visit_2011to2020"]
train_pt_ratio = [4/5]

all_pts = []

In [ ]:
path = "../00_data4pretrain/"
for d in range(len(data_category)):
    print("data_category[d]: ", data_category[d])
    target_folder = path + data_category[d]
    group_folders = glob.glob(target_folder+"/*")
    print("len(group_folders): ", len(group_folders))
    
    for f in range(len(group_folders)):
        if (f+1)%100==0:
            print((f+1), "/", len(group_folders))
        
        group = group_folders[f]
        pateints = glob.glob(group+"/*.txt")
        
        for p in range(len(pateints)):
            if pateints[p] not in all_pts:
                target_category = pateints[p].split("/")[-3]
                target_group = pateints[p].split("/")[-2]
                target_ptnum = pateints[p].split("/")[-1]
                
                ptpath = target_category+"/"+target_group+"/"+target_ptnum
                all_pts.append(ptpath)
        
        
    print("len(all_pts): ", len(all_pts))
    file = open("./pts_"+str(data_category[d])+".txt", "w")
    file.write("\n".join(all_pts))
    file.close()
    
# set heldout dataset
for d in range(len(data_category)):    
    file = open("./pts_"+data_category[d]+".txt", "r")
    all_pts = file.readlines()
    file.close()
    
    all_pts = [ptpath.strip("\n") for ptpath in all_pts]
    
    random.shuffle(all_pts)
    list_heldoutsets = all_pts[:5000]
    file = open("./pts_"+str(data_category[d])+"_heldout.txt", "w")
    file.write("\n".join(list_heldoutsets))
    file.close()
    
    print("heldoutset for finetuning: ", 5000)
    
    
# Move heldout files to here
for d in range(len(data_category)):
    file = open("./pts_"+str(data_category[d])+"_heldout.txt", "r")
    lines = file.readlines()    
    file.close()
    
    for l in range(len(lines)):
        file_in = lines[l].strip("\n")
        file_in = "../00_data4pretrain/"+file_in
        
        # 이동 폴더 만들기
        category  = file_in.split("/")[-3]
        groupname = file_in.split("/")[-2]
        filename  = file_in.split("/")[-1]
        
        file_out = "./"+category+"/"+groupname+"/"
        if not os.path.exists(file_out):
            os.makedirs(file_out)
        
        # 이동
        shutil.move(file_in, file_out+"/"+filename)
        
# print("Done")

# Split into Trainset, Test set
- 5,000 patients
- trainset: 4,000 patients
- testset: 1,000 patients

In [ ]:
import glob
import os
import random

randseed = random.randint(1, 1000)
print("randseed: ", randseed)
rng = random.Random(randseed)

data_category = ["SNUH_visit_2011to2020"]
train_pt_ratio = [4/5]

all_pts = []

In [ ]:
path = "./"
for d in range(len(data_category)):
    file = open("./pts_"+str(data_category[d])+"_heldout.txt", "r")
    heldout_patients = file.readlines()
    file.close()
    
    heldout_patients = [h.strip("\n") for h in heldout_patients]
    
    random.shuffle(heldout_patients)
    
    n_trainset = int(len(heldout_patients)*train_pt_ratio[d])
    n_testset  = len(heldout_patients) - n_trainset
    
    train_pts  = heldout_patients[:n_trainset]
    test_pts   = heldout_patients[n_trainset:]
    
    all_pts.sort()
    
    file = open("./pts_"+str(data_category[d])+"_heldout_train.txt", "w")
    file.write("\n".join(train_pts))
    file.close()
    
    file = open("./pts_"+str(data_category[d])+"_heldout_test.txt", "w")
    file.write("\n".join(test_pts))
    file.close()
    
    print("└output: ./pts_"+str(data_category[d])+".txt")
    print("└number of patients: ", len(heldout_patients))
    print("└number of n_trainset pt: ", len(train_pts))
    print("└number of n_testset pt: ", len(test_pts))
    